# Precipitation Context Analysis for Reclamation Assessment

This notebook demonstrates how to extract and analyze ERA-5 Land precipitation data to provide environmental context for vegetation trends and reclamation monitoring.

**Key Features:**
- Extract monthly precipitation data from ERA-5 Land (April-October growing season)
- Calculate precipitation anomalies compared to 10-year baseline (2010-2019)
- Classify conditions: Extremely Dry, Dry, Normal, Wet, Extremely Wet
- Visualize precipitation patterns and anomalies
- Compare multiple years to identify trends

**Data Source:** ECMWF ERA5-Land Monthly Aggregated - 11km resolution

## Setup and Authentication

In [ ]:
# Import required libraries
import ee
import geemap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from IPython.display import display, HTML

# Import our precipitation analysis module
from precipitation_analysis import PrecipitationAnalyzer

# Set pandas display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [ ]:
# Initialize Earth Engine
try:
    ee.Initialize()
    print("✓ Earth Engine initialized successfully")
except:
    print("Authenticating with Earth Engine...")
    ee.Authenticate()
    ee.Initialize(project="your-project-id")  # Replace with your GEE project ID
    print("✓ Earth Engine initialized successfully")

## Define Area of Interest

In [ ]:
# Example location: Saskatchewan, Canada (matching your reclamation site)
# You can replace this with your actual field/lease boundaries
center_lat = 50.30523
center_lon = -101.80618

# Create a buffer around the point (20km radius for regional precipitation)
point = ee.Geometry.Point([center_lon, center_lat])
region = point.buffer(20000)  # 20km buffer

# Visualize the area
Map = geemap.Map()
Map.centerObject(region, zoom=10)
Map.addLayer(region, {'color': 'blue'}, 'Analysis Region')
Map

## Initialize Precipitation Analyzer

In [ ]:
# Initialize the analyzer with the region and baseline period
precip_analyzer = PrecipitationAnalyzer(
    geometry=region,
    baseline_years=range(2010, 2020)  # 10-year baseline
)

print("Precipitation Analyzer initialized")
print(f"Baseline period: 2010-2019")
print(f"Analysis region: 20km radius around {center_lat:.4f}°N, {center_lon:.4f}°W")

## Calculate Baseline Statistics

In [ ]:
# Calculate baseline precipitation statistics (this may take a minute)
baseline_stats = precip_analyzer.calculate_baseline_statistics()

# Display baseline statistics
baseline_df = pd.DataFrame(baseline_stats).T
baseline_df = baseline_df[['month_name', 'mean', 'median', 'stdev', 'p10', 'p25', 'p75', 'p90']]
baseline_df.columns = ['Month', 'Mean (mm)', 'Median (mm)', 'Std Dev', 'P10', 'P25', 'P75', 'P90']
baseline_df = baseline_df.round(1)

print("\nBaseline Precipitation Statistics (2010-2019):")
display(baseline_df)

## Analyze Recent Growing Seasons

In [ ]:
# Analyze precipitation for recent years (matching your AAFC crop data years)
analysis_years = [2021, 2022, 2023]

for year in analysis_years:
    print(f"\n{'='*60}")
    print(f"PRECIPITATION ANALYSIS FOR {year}")
    print(f"{'='*60}")
    
    # Analyze the growing season
    df = precip_analyzer.analyze_growing_season(year, baseline_stats)
    display(df)
    
    # Create visualization
    precip_analyzer.plot_precipitation_analysis(
        year, 
        baseline_stats,
        save_path=f'output_data/precipitation_analysis_{year}.png'
    )

## Multi-Year Comparison

In [ ]:
# Compare multiple years (2017-2023 to match AAFC data)
comparison_years = list(range(2017, 2024))
comparison_df = precip_analyzer.compare_multiple_years(comparison_years, baseline_stats)

print("\nMulti-Year Precipitation Comparison:")
display(comparison_df)

# Visualize multi-year trends
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Plot 1: Total precipitation by year
ax1 = axes[0]
ax1.bar(comparison_df['Year'], comparison_df['Total Precip (mm)'], color='steelblue', alpha=0.7)
ax1.axhline(y=comparison_df['Normal (mm)'].iloc[0], color='red', linestyle='--', 
            label=f"Normal ({comparison_df['Normal (mm)'].iloc[0]:.0f} mm)")
ax1.set_xlabel('Year')
ax1.set_ylabel('Total Precipitation (mm)')
ax1.set_title('Growing Season Precipitation (April-October)')
ax1.legend()
ax1.grid(True, alpha=0.3)

# Plot 2: Anomaly percentage
ax2 = axes[1]
colors = ['red' if a < 0 else 'green' for a in comparison_df['Anomaly (%)']]
bars = ax2.bar(comparison_df['Year'], comparison_df['Anomaly (%)'], color=colors, alpha=0.7)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.5)
ax2.axhline(y=-15, color='orange', linestyle='--', alpha=0.5, label='Dry threshold')
ax2.axhline(y=15, color='blue', linestyle='--', alpha=0.5, label='Wet threshold')
ax2.set_xlabel('Year')
ax2.set_ylabel('Anomaly (%)')
ax2.set_title('Precipitation Anomaly from Normal')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Add classification labels
for i, (year, classification) in enumerate(zip(comparison_df['Year'], comparison_df['Classification'])):
    ax2.text(year, comparison_df['Anomaly (%)'].iloc[i] + 2, 
             classification.split()[0], ha='center', fontsize=8, rotation=45)

plt.tight_layout()
plt.savefig('output_data/multi_year_precipitation_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

## Integration with Reclamation Analysis

The precipitation context can now be integrated with your reclamation analysis to:
1. **Explain vegetation anomalies**: Dry years may show lower productivity unrelated to reclamation success
2. **Adjust expectations**: Wet years may show enhanced growth that masks poor reclamation
3. **Normalize comparisons**: Account for precipitation when comparing lease vs reference fields
4. **Track recovery**: Separate weather effects from actual reclamation progress

In [ ]:
# Example: Create a summary for integration with reclamation assessment
def generate_precipitation_context(year, precip_analyzer, baseline_stats):
    """
    Generate a precipitation context summary for a specific year.
    """
    df = precip_analyzer.analyze_growing_season(year, baseline_stats)
    season_row = df[df['Month'] == 'SEASONAL TOTAL'].iloc[0]
    
    context = {
        'year': year,
        'total_precip_mm': season_row['Precipitation (mm)'],
        'anomaly_percent': season_row['Anomaly (%)'],
        'classification': season_row['Classification'],
        'vegetation_impact': 'neutral'
    }
    
    # Determine vegetation impact
    if season_row['Anomaly (%)'] < -30:
        context['vegetation_impact'] = 'severe_negative'
        context['impact_description'] = 'Severe drought stress likely limiting vegetation growth'
    elif season_row['Anomaly (%)'] < -15:
        context['vegetation_impact'] = 'moderate_negative'
        context['impact_description'] = 'Moderate moisture deficit affecting vegetation productivity'
    elif season_row['Anomaly (%)'] > 30:
        context['vegetation_impact'] = 'potential_negative'
        context['impact_description'] = 'Excessive moisture may cause flooding or disease stress'
    elif season_row['Anomaly (%)'] > 15:
        context['vegetation_impact'] = 'positive'
        context['impact_description'] = 'Above-normal moisture supporting enhanced vegetation growth'
    else:
        context['vegetation_impact'] = 'neutral'
        context['impact_description'] = 'Normal moisture conditions for typical vegetation growth'
    
    return context

# Generate context for each year
print("\nPrecipitation Context for Reclamation Assessment:")
print("="*60)

for year in [2021, 2022, 2023]:
    context = generate_precipitation_context(year, precip_analyzer, baseline_stats)
    print(f"\n{year}:")
    print(f"  Classification: {context['classification']}")
    print(f"  Anomaly: {context['anomaly_percent']:.1f}%")
    print(f"  Impact: {context['impact_description']}")

## Export Results for Integration

In [ ]:
# Save precipitation data for integration with reclamation analysis
import json
import os

# Ensure output directory exists
os.makedirs('output_data', exist_ok=True)

# Prepare data for export
export_data = {
    'baseline_period': '2010-2019',
    'analysis_region': {
        'center_lat': center_lat,
        'center_lon': center_lon,
        'buffer_km': 20
    },
    'yearly_analysis': {}
}

for year in comparison_years:
    context = generate_precipitation_context(year, precip_analyzer, baseline_stats)
    export_data['yearly_analysis'][year] = context

# Save to JSON
with open('output_data/precipitation_context.json', 'w') as f:
    json.dump(export_data, f, indent=2)

# Save comparison DataFrame to CSV
comparison_df.to_csv('output_data/precipitation_comparison.csv', index=False)

print("✓ Precipitation analysis data exported to:")
print("  - output_data/precipitation_context.json")
print("  - output_data/precipitation_comparison.csv")
print("  - Individual year visualizations in output_data/")